# 다트 크롤링
- 크롤링할 컬럼: 기업명, 주소, 업종분류, 설립연도, 기업홈페이지URL
- 절차
1. 다트 공시현황 사이트 접속
2. 기업명 타이핑하여 공시정보 접속
3. 내용 크롤링
4. 타이핑 되어있는 기업명 제거
5. concat
6. 다음 기업명 타이핑하여 공시정보 접속

위 절차를 반복합니다.

In [ ]:
import pandas as pd
# 정적크롤링
from bs4 import BeautifulSoup
import requests
import re
# 동적크롤링
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.keys import Keys

In [ ]:
# 파일 불러오기: 불러올 파일을 현재 폴더에 넣어두세요.
df = pd.read_csv('./programmers.csv')  # 파일명만 변경하여 그대로 돌리시면 됩니다.

In [ ]:
# 임시 프레임
dart_res = pd.DataFrame(columns=['기업명','주소','업종분류','설립연도','기업홈페이지URL'])

In [ ]:
# 크롤링 프로그램
url = 'https://dart.fss.or.kr/dsae001/main.do#none'

driver= webdriver.Chrome()
driver.get(url)

driver.find_element(By.XPATH, '//*[@id="textCrpNm"]').clear()

for i in df['기업명']:
    try:
        driver.find_element(By.XPATH, '//*[@id="textCrpNm"]').send_keys(i)  # 키워드 작성
        driver.find_element(By.XPATH, '//*[@id="rightWrap"]/div[1]/div[1]/ul/li[3]/a').click()
        driver.find_element(By.XPATH, '//*[@id="rightWrap"]/div[1]/div[1]/ul/li[3]/a').click()
        time.sleep(.1)
        driver.find_element(By.XPATH, '//*[@id="corpTabel"]/tbody/tr/td[1]/span/a').click()

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 기업명
        corp_name = i
        # 주소
        address = driver.find_element(By.XPATH, '//*[@id="corpDetailTabel"]/tbody/tr[9]/td').text
        # 업종명
        sector = driver.find_element(By.XPATH, '//*[@id="corpDetailTabel"]/tbody/tr[14]/td').text
        # 설립일
        corp_year = driver.find_element(By.XPATH, '//*[@id="corpDetailTabel"]/tbody/tr[15]/td').text
        # 기업 홈페이지URL
        corp_url = driver.find_element(By.XPATH, '//*[@id="homePage"]').text

        dart_temp = pd.DataFrame([corp_name, address, sector, corp_year, corp_url],
                                  index=['기업명','주소','업종분류','설립연도','기업홈페이지URL']).T
        dart_res = pd.concat([dart_res, dart_temp])

        # 키워드 지우기
        driver.find_element(By.XPATH, '//*[@id="textCrpNm"]').clear()
        time.sleep(.1)
        
    except:
        driver.find_element(By.XPATH, '//*[@id="textCrpNm"]').clear()

driver.quit()

In [ ]:
dart_res.to_csv('./dart_result.csv')

# 코드 수정
- 기업명 키워드로 나온 모든 기업정보 크롤링

In [ ]:
import pandas as pd
# 정적크롤링
from bs4 import BeautifulSoup
import requests
import re
# 동적크롤링
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.keys import Keys

In [ ]:
# 파일 불러오기: 불러올 파일을 현재 폴더에 넣어두세요.
df = pd.read_csv('./programmers.csv')  # 파일명만 변경하여 그대로 돌리시면 됩니다.
df_corp_name = df['기업명']  # 기업명 불러오기

In [ ]:
# 임시 프레임
dart_res = pd.DataFrame(columns=['기업명', '다트기업명','주소','업종분류','설립연도','기업 홈페이지 URL'])

In [ ]:
# 크롤링 프로그램
url = 'https://dart.fss.or.kr/dsae001/main.do#none'

driver = webdriver.Chrome()
driver.get(url)

for co in df['기업명']:
    try:
        # 기업명 입력 및 검색 로직
        driver.find_element(By.XPATH, '//*[@id="textCrpNm"]').send_keys(co)
        # 검색 버튼 클릭 등의 로직
        driver.find_element(By.XPATH, '//*[@id="rightWrap"]/div[1]/div[1]/ul/li[3]/a').click()
        time.sleep(.1)

        for i in range(100):
            try:
                driver.find_element(By.XPATH, f'//*[@id="corpTabel"]/tbody/tr[{i}]/td[1]/span/a').click()
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                    
                # 다트 내 기업명
                corp_name = driver.find_element(By.XPATH, '//*[@id="corpDetailTabel"]/tbody/tr[3]/td').text
                # 주소
                address = driver.find_element(By.XPATH, '//*[@id="corpDetailTabel"]/tbody/tr[9]/td').text
                # 업종명
                sector = driver.find_element(By.XPATH, '//*[@id="corpDetailTabel"]/tbody/tr[14]/td').text
                # 설립일
                corp_year = driver.find_element(By.XPATH, '//*[@id="corpDetailTabel"]/tbody/tr[15]/td').text
                # 기업 홈페이지URL
                corp_url = driver.find_element(By.XPATH, '//*[@id="homePage"]').text
                
                dart_temp = pd.DataFrame([[co, corp_name, address, sector, corp_year, corp_url]],
                                            columns=['기업명', '다트기업명', '주소', '업종분류', '설립연도', '기업 홈페이지 URL'])
                dart_res = pd.concat([dart_res, dart_temp])
            except:
                # 예외 처리 및 검색창 지우기
                driver.find_element(By.XPATH, '//*[@id="textCrpNm"]').clear()

    except:
        # 예외 처리 및 검색창 지우기
        driver.find_element(By.XPATH, '//*[@id="textCrpNm"]').clear()

driver.quit()

In [ ]:
dart_res.to_csv('./dart.result.csv')

# 다트 전처리

In [ ]:
dart_res.설립연도 = dart_res.설립연도.apply(lambda x: ''.join(re.findall(r'\d\d\d\d', x)))  # 설립연도
dart_res.주소 = dart_res.주소.apply(lambda x: ' '.join(re.findall(r'\b(\w+[구시도])\b', x)))  # 주소
dart_res.설립연도 = dart_res.설립연도.astype(int)  # 타입변경
dart_res.reset_index(drop=True, inplace=True)  # 인덱스 재설정

In [ ]:
dart_res